In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
from utils import ProteinInteractionGraph
from model import ProteinInteractionDataPreparation

c:\Users\AHMED MRABET\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create a graph of protein interactions
interacation_graph = ProteinInteractionGraph(negative_interaction_method='most_distant')

In [4]:
# Convert the graph to a format suitable for the model
data_preparation = ProteinInteractionDataPreparation(interacation_graph.graph, interacation_graph.adjacency_matrix, interacation_graph.node_features, batch_size=64, train_ratio=0.8, val_ratio=0.1)
node_features_tensor, train_loader, val_loader, test_loader = data_preparation.get_data_loaders()

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import GNN

In [6]:
num_node_features = node_features_tensor.shape[1]
model = GNN(num_node_features, hidden_channels=64, out_channels=1)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node_features_tensor = node_features_tensor.to(device)
model = model.to(device)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [9]:
epochs = 50

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for i, (edge_indices, edge_labels, batch) in enumerate(train_loader):
        edge_indices, edge_labels, batch = edge_indices.to(device), edge_labels.to(device), batch.to(device)
        optimizer.zero_grad()
        out = model(node_features_tensor, edge_indices.t(), batch)
        loss = criterion(out, edge_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for edge_indices, edge_labels, batch in val_loader:
            edge_indices, edge_labels, batch = edge_indices.to(device), edge_labels.to(device), batch.to(device)
            out = model(node_features_tensor, edge_indices.t(), batch)
            loss = criterion(out, edge_labels)
            val_loss += loss.item()

    print(f"Epoch: {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")


RuntimeError: The expanded size of the tensor (10276) must match the existing size (64) at non-singleton dimension 0.  Target sizes: [10276, 64].  Tensor sizes: [64, 1]

In [ ]:
# Test loop
model.eval()
test_loss = 0
with torch.no_grad():
    for edge_indices, edge_labels, batch in test_loader:
        edge_indices, edge_labels, batch = edge_indices.to(device), edge_labels.to(device), batch.to(device)
        out = model(node_features_tensor, edge_indices.t(), batch)
        loss = criterion(out, edge_labels)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")
